<a href="https://colab.research.google.com/github/jantuitman/musicexperiments/blob/main/scales_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scales finder

This notebook prints scales that match chords or notes that you supply.

In [8]:
class Note:
    NOTES = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    NOTE_VALUES = {note: i for i, note in enumerate(NOTES)}
    VALUE_NOTES = {i: note for i, note in enumerate(NOTES)}

    def __init__(self, name):
        self.name = name
        self.value = self.NOTE_VALUES[name]

    def interval_to(self, other):
        return (other.value - self.value) % 12

    def __repr__(self):
        return self.name


In [28]:
class Scale:
    def __init__(self, root, intervals, name, mode=0, scale_type=""):
        self.root = Note(root)
        self.intervals = intervals
        self.name = name
        self.scale_type = scale_type
        self.systematic_name = f"{root} {scale_type}, mode +{mode}"
        self.notes = [self.root.name] + [Note.VALUE_NOTES[(self.root.value + interval) % 12] for interval in intervals]

    def contains(self, notes):
        return all(note.name in self.notes for note in notes)

    def __repr__(self):
        notes_str = ', '.join([str(note) for note in self.notes])
        return f"{self.name} ({self.systematic_name}): Notes({notes_str})"

    def get_intervals_markdown(self, chord_notes):
        intervals_md = []
        for interval, note in zip([0] + self.intervals, self.notes):
            if note in [n.name for n in chord_notes]:
                intervals_md.append(f"- **{interval}**: **{note}**")
            else:
                intervals_md.append(f"- {interval}: {note}")
        return '\n'.join(intervals_md)


In [10]:
# Define the ScaleCollection class
class ScaleCollection:
    def __init__(self):
        self.scales = []

    def add_scale(self, scale):
        self.scales.append(scale)

    def match_scales(self, chord_notes):
        matching_scales = []
        for scale in self.scales:
            if scale.contains(chord_notes):
                matching_scales.append(scale)
        return matching_scales

In [19]:
def add_scales_for_all_roots(scale_collection, intervals, name, scale_type, mode=0):
    for root in Note.NOTES:
        scale_collection.add_scale(Scale(root, intervals, f"{root} {name}", mode, scale_type))


In [34]:
# Initialize the scale collection with common scales
scale_collection = ScaleCollection()

# Define the intervals for the Major scale and its modes
major_intervals = [0, 2, 4, 5, 7, 9, 11]
major_mode_names = ["Ionian", "Dorian", "Phrygian", "Lydian", "Mixolydian", "Aeolian", "Locrian"]

# Define the intervals for the Harmonic Minor scale and its modes
harmonic_minor_intervals = [0, 2, 3, 5, 7, 8, 11]
harmonic_minor_mode_names = ["Harmonic Minor", "Locrian #6", "Ionian #5", "Dorian #4", "Phrygian Dominant", "Lydian #2", "Altered Dominant bb7"]

# Define the intervals for the Melodic Minor scale and its modes
melodic_minor_intervals = [0, 2, 3, 5, 7, 9, 11]
melodic_minor_mode_names = ["Melodic Minor (Ascending)", "Dorian b2", "Lydian Augmented", "Lydian Dominant", "Mixolydian b6", "Locrian #2", "Altered Scale"]

# Define the intervals for the Melodic Major scale and its modes
melodic_major_intervals = [0, 2, 4, 5, 7, 8, 11]
melodic_major_mode_names = ["Melodic Major", "Dorian b5", "Phrygian b4", "Lydian b3", "Mixolydian b2", "Locrian b1", "Superlocrian"]

# Function to rotate and normalize intervals to get modes
def get_mode_intervals(intervals, mode):
    root = intervals[mode]
    rotated = [(interval - root) % 12 for interval in intervals]
    rotated.sort()
    return rotated

# Add Major scale and its modes
for mode, name in enumerate(major_mode_names):
    intervals = get_mode_intervals(major_intervals, mode)
    add_scales_for_all_roots(scale_collection, intervals[1:], name, "Major", mode)

# Add Harmonic Minor scale and its modes
for mode, name in enumerate(harmonic_minor_mode_names):
    intervals = get_mode_intervals(harmonic_minor_intervals, mode)
    add_scales_for_all_roots(scale_collection, intervals[1:], name, "Harmonic Minor", mode)

# Add Melodic Minor scale and its modes
for mode, name in enumerate(melodic_minor_mode_names):
    intervals = get_mode_intervals(melodic_minor_intervals, mode)
    add_scales_for_all_roots(scale_collection, intervals[1:], name, "Melodic Minor", mode)

# Add Melodic Major scale and its modes
for mode, name in enumerate(melodic_major_mode_names):
    intervals = get_mode_intervals(melodic_major_intervals, mode)
    add_scales_for_all_roots(scale_collection, intervals[1:], name, "Melodic Major", mode)


In [33]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# Define the text boxes
text_box_notes = widgets.Text(
    value='C E G',
    placeholder='Enter note names separated by space',
    description='Chord Notes:',
    disabled=False
)

text_box_root = widgets.Text(
    value='',
    placeholder='Enter root note (optional)',
    description='Root Note:',
    disabled=False
)

# Define a button
button_find_scales = widgets.Button(description="Find Scales")

# Define the output area
output_notes = widgets.Output()

result_markdown='';

# Define the event handler
def find_scales(b):
    with output_notes:
        clear_output()
        notes = text_box_notes.value.split()
        chord_notes = [Note(note) for note in notes]
        root_note = text_box_root.value.strip()
        matching_scales = scale_collection.match_scales(chord_notes)
        if root_note:
          root_note_value = Note.NOTE_VALUES.get(root_note, None)
          if root_note_value is not None:
            matching_scales = [scale for scale in matching_scales if scale.root.value == root_note_value]

        result_markdown = [f"**Entered chord notes:** {notes}"]
        if root_note is not None:
            result_markdown.append(f"**Matching scales with root note {root_note}:**")
        else:
            result_markdown.append("**Matching scales:**")

        for scale in matching_scales:
            scale_md = f"{scale}\n**Intervals:**\n{scale.get_intervals_markdown(chord_notes)}"
            result_markdown.append(scale_md)


# Attach the event handler to the button
button_find_scales.on_click(find_scales)

# Display the text boxes, button, and output area
display(text_box_notes, text_box_root, button_find_scales, output_notes, Markdown("\n\n".join(result_markdown))


Text(value='C E G', description='Chord Notes:', placeholder='Enter note names separated by space')

Text(value='', description='Root Note:', placeholder='Enter root note (optional)')

Button(description='Find Scales', style=ButtonStyle())

Output()